## Uvod

Ta lekcija bo zajemala:
- Kaj je klic funkcije in njene uporabe
- Kako ustvariti klic funkcije z uporabo OpenAI
- Kako integrirati klic funkcije v aplikacijo

## Cilji učenja

Po zaključku te lekcije boste znali in razumeli:

- Namen uporabe klica funkcije
- Nastavitev klica funkcije z uporabo OpenAI storitve
- Oblikovanje učinkovitih klicev funkcij za uporabo v vaši aplikaciji


## Razumevanje klicev funkcij

Za to lekcijo želimo zgraditi funkcijo za naš izobraževalni startup, ki uporabnikom omogoča uporabo klepetalnega robota za iskanje tehničnih tečajev. Priporočali bomo tečaje, ki ustrezajo njihovi ravni znanja, trenutni vlogi in tehnologiji, ki jih zanima.

Za dokončanje bomo uporabili kombinacijo:
 - `OpenAI` za ustvarjanje klepetalnega doživetja za uporabnika
 - `Microsoft Learn Catalog API` za pomoč uporabnikom pri iskanju tečajev na podlagi njihove zahteve
 - `Function Calling` za prevzem uporabnikovega poizvedbe in pošiljanje funkciji za izvedbo API zahteve.

Za začetek si poglejmo, zakaj bi sploh želeli uporabiti klic funkcije:

print("Sporočila v naslednjem zahtevku:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # pridobi nov odgovor od GPT, kjer lahko vidi odgovor funkcije


print(second_response.choices[0].message)


### Zakaj klic funkcij

Če ste opravili katero koli drugo lekcijo v tem tečaju, verjetno razumete moč uporabe velikih jezikovnih modelov (LLM). Upamo, da lahko vidite tudi nekatere njihove omejitve.

Klic funkcij je funkcija storitve OpenAI, zasnovana za reševanje naslednjih izzivov:

Nekonsistentno oblikovanje odgovorov:
- Pred uvedbo klica funkcij so bili odgovori velikega jezikovnega modela nestrukturirani in nekonsistentni. Razvijalci so morali pisati zapleteno kodo za preverjanje, da so obravnavali vsako različico izhoda.

Omejena integracija z zunanjimi podatki:
- Pred to funkcijo je bilo težko vključiti podatke iz drugih delov aplikacije v kontekst klepeta.

S standardizacijo oblik odgovorov in omogočanjem nemotene integracije z zunanjimi podatki klic funkcij poenostavi razvoj in zmanjša potrebo po dodatni logiki preverjanja.

Uporabniki niso mogli dobiti odgovorov, kot je "Kakšno je trenutno vreme v Stockholmu?". To je zato, ker so bili modeli omejeni na čas, ko so bili podatki usposobljeni.

Poglejmo spodnji primer, ki ponazarja ta problem:

Recimo, da želimo ustvariti bazo podatkov o študentih, da jim lahko predlagamo pravi tečaj. Spodaj imamo dva opisa študentov, ki sta zelo podobna v podatkih, ki jih vsebujeta.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Želimo to poslati LLM-ju, da obdela podatke. To lahko kasneje uporabimo v naši aplikaciji za pošiljanje na API ali shranjevanje v podatkovno bazo.

Ustvarimo dva enaka poziva, v katerih LLM-ju naročimo, katere informacije nas zanimajo:


Želimo to poslati LLM-ju, da razčleni dele, ki so pomembni za naš izdelek. Tako lahko ustvarimo dva enaka poziva za usmerjanje LLM-ja:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po ustvarjanju teh dveh pozivov jih bomo poslali LLM z uporabo `openai.ChatCompletion`. Poziv shranimo v spremenljivko `messages` in dodelimo vlogo `user`. To je, da posnemamo sporočilo uporabnika, ki ga piše chatbotu.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Zdaj lahko pošljemo oba zahtevka LLM-ju in pregledamo odgovor, ki ga prejmemo.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Čeprav so pozivi enaki in so opisi podobni, lahko dobimo različne formate lastnosti `Grades`.

Če zgornjo celico zaženete večkrat, je lahko format `3.7` ali `3.7 GPA`.

To je zato, ker LLM vzame nestrukturirane podatke v obliki napisanega poziva in vrne tudi nestrukturirane podatke. Potrebujemo strukturiran format, da vemo, kaj pričakovati pri shranjevanju ali uporabi teh podatkov.

Z uporabo funkcijskega klica lahko zagotovimo, da prejmemo nazaj strukturirane podatke. Pri uporabi funkcijskega klica LLM dejansko ne kliče ali izvaja nobenih funkcij. Namesto tega ustvarimo strukturo, ki jo LLM sledi za svoje odgovore. Nato te strukturirane odgovore uporabimo, da vemo, katero funkcijo naj zaženemo v naših aplikacijah.


![Diagram poteka klica funkcije](../../../../translated_images/sl/Function-Flow.083875364af4f4bb.webp)


Nato lahko vzamemo, kar funkcija vrne, in to pošljemo nazaj LLM-ju. LLM bo nato odgovoril z uporabo naravnega jezika, da odgovori na uporabnikovo vprašanje.


### Primeri uporabe klicev funkcij

**Klicanje zunanjih orodij**  
Chatboti so odlični pri zagotavljanju odgovorov na vprašanja uporabnikov. Z uporabo klicev funkcij lahko chatboti uporabijo sporočila uporabnikov za izvedbo določenih nalog. Na primer, študent lahko prosi chatbota: "Pošlji e-pošto mojemu inštruktorju, da potrebujem več pomoči pri tem predmetu". To lahko sproži klic funkcije `send_email(to: string, body: string)`.

**Ustvarjanje API ali poizvedb v bazi podatkov**  
Uporabniki lahko najdejo informacije z uporabo naravnega jezika, ki se pretvori v oblikovano poizvedbo ali API zahtevo. Primer tega je učitelj, ki vpraša: "Kdo so študenti, ki so opravili zadnjo nalogo", kar lahko sproži klic funkcije z imenom `get_completed(student_name: string, assignment: int, current_status: string)`.

**Ustvarjanje strukturiranih podatkov**  
Uporabniki lahko vzamejo blok besedila ali CSV in uporabijo LLM za izvleček pomembnih informacij iz njega. Na primer, študent lahko pretvori Wikipedijin članek o mirovnih sporazumih v AI kartice za učenje. To se lahko izvede z uporabo funkcije `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.


## 2. Ustvarjanje vašega prvega klica funkcije

Postopek ustvarjanja klica funkcije vključuje 3 glavne korake:  
1. Klic API-ja Chat Completions z seznamom vaših funkcij in uporabniškim sporočilom  
2. Branje odziva modela za izvedbo dejanja, npr. izvedbo funkcije ali klica API-ja  
3. Izvedba še enega klica API-ja Chat Completions z odzivom vaše funkcije, da uporabite te informacije za ustvarjanje odgovora uporabniku.


![Potek klica funkcije](../../../../translated_images/sl/LLM-Flow.3285ed8caf4796d7.webp)


### Elementi klica funkcije 

#### Vnos uporabnika 

Prvi korak je ustvariti uporabniško sporočilo. To je mogoče dinamično dodeliti z vrednostjo besedilnega vnosa ali pa lahko tukaj dodelite vrednost. Če je to vaš prvič, da delate z API-jem za Chat Completions, moramo definirati `role` in `content` sporočila. 

`role` je lahko bodisi `system` (ustvarjanje pravil), `assistant` (model) ali `user` (končni uporabnik). Za klic funkcije bomo to dodelili kot `user` in primer vprašanja. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Ustvarjanje funkcij.

Nato bomo definirali funkcijo in parametre te funkcije. Tukaj bomo uporabili samo eno funkcijo, imenovano `search_courses`, vendar lahko ustvarite več funkcij.

**Pomembno** : Funkcije so vključene v sistemsko sporočilo za LLM in bodo vključene v količino razpoložljivih žetonov, ki jih imate na voljo.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicije** 

Struktura definicije funkcije ima več nivojev, vsak s svojimi lastnostmi. Tukaj je razčlenitev gnezdene strukture:

**Lastnosti funkcije na najvišji ravni:**

`name` - Ime funkcije, ki jo želimo poklicati. 

`description` - To je opis, kako funkcija deluje. Pomembno je, da je tukaj specifičen in jasen. 

`parameters` - Seznam vrednosti in formata, ki jih želite, da model ustvari v svojem odgovoru. 

**Lastnosti objekta parametrov:**

`type` - Podatkovni tip objekta parametrov (običajno "object")

`properties` - Seznam specifičnih vrednosti, ki jih bo model uporabil za svoj odgovor. 

**Lastnosti posameznega parametra:**

`name` - Implicitno določen s ključem lastnosti (npr. "role", "product", "level")

`type` - Podatkovni tip tega specifičnega parametra (npr. "string", "number", "boolean") 

`description` - Opis specifičnega parametra. 

**Neobvezne lastnosti:**

`required` - Polje, ki navaja, kateri parametri so potrebni za uspešno izvedbo klica funkcije. 


### Klic funkcije  
Po definiranju funkcije jo moramo zdaj vključiti v klic API-ja za Chat Completion. To naredimo tako, da v zahtevo dodamo `functions`. V tem primeru `functions=functions`.  

Obstaja tudi možnost nastavitve `function_call` na `auto`. To pomeni, da bomo pustili LLM, da odloči, katera funkcija naj bo poklicana na podlagi uporabnikovega sporočila, namesto da bi jo določili sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Zdaj si poglejmo odgovor in kako je oblikovan:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Lahko vidite, da je ime funkcije poklicano in iz uporabnikovega sporočila je LLM uspel najti podatke, ki ustrezajo argumentom funkcije.


## 3. Integracija klicev funkcij v aplikacijo. 


Ko smo preizkusili oblikovan odgovor iz LLM, ga lahko zdaj integriramo v aplikacijo. 

### Upravljanje poteka 

Za integracijo tega v našo aplikacijo naredimo naslednje korake: 

Najprej izvedimo klic na OpenAI storitve in sporočilo shranimo v spremenljivko z imenom `response_message`. 


In [ ]:
response_message = response.choices[0].message

Zdaj bomo definirali funkcijo, ki bo poklicala Microsoft Learn API za pridobitev seznama tečajev:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Kot najboljšo prakso bomo nato preverili, ali model želi poklicati funkcijo. Po tem bomo ustvarili eno od razpoložljivih funkcij in jo ujemali s funkcijo, ki se kliče.  
Nato bomo vzeli argumente funkcije in jih preslikali na argumente iz LLM.

Na koncu bomo dodali sporočilo o klicu funkcije in vrednosti, ki jih je vrnilo sporočilo `search_courses`. To LLM-ju zagotovi vse informacije, ki jih potrebuje za 
odziv uporabniku v naravnem jeziku.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Zdaj bomo poslali posodobljeno sporočilo LLM, da bomo prejeli odgovor v naravnem jeziku namesto odgovora v obliki JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Izziv s kodo

Odlično delo! Za nadaljevanje učenja o klicanju funkcij OpenAI lahko ustvarite: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Več parametrov funkcije, ki lahko pomagajo učencem najti več tečajev. Na voljo API parametre lahko najdete tukaj:  
 - Ustvarite še en klic funkcije, ki od učenca zahteva več informacij, na primer njihov materni jezik  
 - Ustvarite obravnavo napak, ko klic funkcije in/ali klic API ne vrne nobenih ustreznih tečajev  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda, ne odgovarjamo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
